In [1]:
def get_instruction():
    instruction = "你是电商领域的专家，请你判断以下商品评论是正向的还是负向的。如果是正向的，请回答1，如果是负向的，请回答0。"
    return instruction

def get_input(sentence):
    input = f'商品评论：{sentence}'
    return input

In [3]:
import json
import os
from openai import OpenAI
from transformers.utils.versions import require_version
import pandas as pd
from tqdm import tqdm

def predict(port, df_pth):
    client = OpenAI(
        api_key="0",
        base_url="http://localhost:{}/v1".format(port)
    )

    output=[]
    count=0
    right=0
    true_label_count={}
    predict_label_count = {}

    df = pd.read_csv(df_pth)
    df = df.fillna('')
    df = df[df['label']!='']
    df['label'] = df['label'].apply(lambda x: int(x))
    for index, row in tqdm(df.iterrows()):
        messages = []
        input = get_input(row['sentence'])
        instruction = get_instruction()
        messages.append({"role": "user",
                            "content": instruction + input})

        result = client.chat.completions.create(messages=messages, model="test",temperature=0)
        try:
            predict_label = json.loads(result.choices[0].message.content.strip())
        except Exception as e:
            print(e)
            print(result.choices[0].message.content)
            predict_label = "-1"

        label = row['label']
        count += 1
        if label not in true_label_count:
            true_label_count[label] = 1
        else:
            true_label_count[label] += 1

        if predict_label not in predict_label_count:
            predict_label_count[predict_label] = 1
        else:
            predict_label_count[predict_label] += 1

        if label == predict_label:
            right += 1

    print("数据总数:",count)
    print("label的分布情况:",true_label_count)
    print("预测label的分布情况:", predict_label_count)
    print("acc:",right/count)


predict(port=8000, df_pth='./data/DAMO_NLP_jd/dev.csv')

0it [00:00, ?it/s]

4988it [07:23, 11.25it/s]

数据总数: 4988
label的分布情况: {1: 2441, 0: 2547}
预测label的分布情况: {1: 2407, 0: 2581}
acc: 0.9202085004009624
